**このコードでは、Prophetを用いて1日と1週間単位の二つの値で予測するという作業を行う**

前提として、

*  Data_preprocessing_1とData_preprocessing_city&AQI_2、Data_feature&Missing_value_imputation_3を動かし、訓練・検証・テストデータとして用いるデータがcsvとして保存されていること

が条件となる。

**参考資料**

*   [Optunaを用いたProphetモデルのコード参考サイト](https://book.st-hakky.com/data-science/prophet-model-predicts/)

*  [Prophetの公式サイト](https://facebook.github.io/prophet/)





**注意事項**

※ 作成者はGoogle drive内で作業をしていると同時に、ゼミというフォルダーの中の公開ソースというフォルダーで作業していることからpathの変更は必要不可欠である

# 初期設定

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#ハイパーパラメータで使用するoptunaをインストール
! pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.5 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error
from prophet import Prophet
import optuna

In [ ]:
#データフレームを読み込み
working_dir = '/content/drive/MyDrive/ゼミ/公開ソース' #　※※自分で作成したフォルダパスが異なる場合こちらを変更してください。※※
path_df_city = f'{working_dir}/data/traninig_validation_test_data'

In [ ]:
#TokyoとDelhiのデータフレームを読み込む
for i in (os.listdir(path_df_city)):
  name = i.partition('_')[0]
  df = pd.read_csv(f"{path_df_city}/{i}",index_col=0,parse_dates=True)
  print(name)
  exec("{}=df".format(name))

Tokyo
Delhi


In [ ]:
#目的変数の絞り込み
ov= ["AQI_total","pm25","pm10","so2","o3","no2","co"]

In [ ]:
#MAPE値を保存するデータフレーム作成
df_evaluation = pd.DataFrame(columns=["prophet_week","prophet_day"])

In [ ]:
#グラフとデータを保存する場所を指定
graph_path = f"{working_dir}/result/graph/Prophet"
data_path = f"{working_dir}/result/data/Prophet"

In [ ]:
#MAPE値を算出する関数
def mape(predict, observed):
  absolute_diff_percentage =  abs( (predict - observed) / observed)
  sum_abs_diff = sum(absolute_diff_percentage)
  mape = sum_abs_diff / len(predict)
  return mape

In [ ]:
#予測値のデータフレームを予測する場所を指定
data_path = f"{working_dir}/result/data/Prophet"

In [ ]:
#時系列のグラフを保存する場所を指定
graph_path = f"{working_dir}/result/graph/Prophet"

# Prophetによる予測(1週間単位と1日単位)



*   訓練・検証・テストデータの三つに分けつつ最適な予測モデルの構築を行っていく
*   検証・テストデータをそれぞれ約一年分、それ以外のデータを訓練データとして使用する流れ

※ Data _feature&Missing _value _imputation_3にて、TokyoとDelhiのデータがそれぞれ7日と1日欠損しているが、どちらも訓練データ要素の欠損になるため、検証データとテストデータは1日単位と1週間単位それぞれ365と52とデータの数を定めている

In [ ]:
#1週間でのprophetを用いた予測を行う関数
def model_prophet_week(df,city_name):
  for x in ov:
    #ここで1週間単位に変更(7日間の平均値を1週間の値と設定)
    df = df.resample("W").mean()
    #prophetでは、目的変数をy,日付をdsと列名を指定しなければならないためここで行う
    df_y = df.rename(columns={x:'y'})
    dff = df_y.rename_axis("ds")

    #訓練データとテストデータを作成
    y_train =dff[["y"]][:-104].reset_index()
    y_test =dff[["y"]][-52:].reset_index()

    #ハイパーパラメータで用いる検証データ作成
    y_val = dff[["y"]][-104:-52].reset_index()


#optunaを用いたハイパーパラメータ設定
#------------------------------------------------
    def objective(trial):
      params = {'changepoint_prior_scale' :
                  trial.suggest_uniform('changepoint_prior_scale',
                                        0.001,0.5
                                        ),
                'seasonality_prior_scale' :
                  trial.suggest_uniform('seasonality_prior_scale',
                                        0.01,10
                                        ),
                'seasonality_mode' :
                  trial.suggest_categorical('seasonality_mode',
                                            ['additive', 'multiplicative']
                                            ),
                'changepoint_range' :
                    trial.suggest_discrete_uniform('changepoint_range',
                                                  0.8, 0.95,
                                                  0.001),
                'n_changepoints' :
                    trial.suggest_int('n_changepoints',
                                      20, 35),
              }
      m = Prophet(**params)
      m.fit(y_train)
      df_future = m.make_future_dataframe(periods=len(y_val),freq='W')
      df_pred = m.predict(df_future)
      preds = df_pred.tail(len(y_val))
      val_rmse = np.sqrt(mean_squared_error(y_val.y, preds.yhat))
      return val_rmse



    # ハイパーパラメータの探索の実施
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=100)
    m = Prophet(**study.best_params)
    m.fit(y_train)

    #予測する範囲を検証データとテストデータの二つの長さの合計としてperiodsで指定
    future = m.make_future_dataframe(periods =len(y_val)+len(y_test),freq='W')
    pred = m.predict(future)

    #テストデータの範囲内での観測値と予測値に絞る
    result_df = pd.merge(pred[["ds","yhat"]].astype(str),y_test.astype(str),on="ds",how="inner")

    #グラフに表示するための簡易的なデータフレーム設定
    result_df = result_df.rename(columns={"ds":"Date","y":x,"yhat":f"predicted_value_{x}"})
    result_df["Date"] = pd.to_datetime(result_df["Date"])
    result_df[x] = result_df[x].astype(float)
    result_df[f"predicted_value_{x}"] = result_df[f"predicted_value_{x}"].astype(float)
    result = result_df.set_index("Date")

    #各目的変数ごとの観測値と予測値のデータフレームをcsvとして保存
    result.to_csv(f"{data_path}/{city_name}_{x}_week.csv")
    #mape値の抽出
    mape_df = str(str(round(mape(result_df[f"predicted_value_{x}"], result_df[x])*100,2))+"%")
    #計算したmape値をデータフレームに書き出し
    df_evaluation.at[f"{city_name}_{x}","prophet_week"] = mape_df


    #グラフの描画と保存
    result_df.set_index("Date").plot()
    plt.xlabel('Date')
    plt.ylabel(x)
    plt.title(f'{city_name}(2022/11/13~2023/11/5){x}_value_prophet_week')
    plt.legend()
    plt.savefig(f'{graph_path}/{city_name}_{x}_prophet_pred_week.png',bbox_inches='tight')
    plt.show()

  return result

In [ ]:
#Delhi
model_prophet_week(Delhi,"Delhi")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#Tokyo
model_prophet_week(Tokyo,"Tokyo")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#MAPE値の確認
df_evaluation

,prophet_week,prophet_day
Delhi_AQI_total,7.8%,NaN
Delhi_pm25,13.89%,NaN
Delhi_pm10,26.35%,NaN
Delhi_so2,16.86%,NaN
Delhi_o3,15.8%,NaN
Delhi_no2,15.3%,NaN
Delhi_co,16.46%,NaN
Tokyo_AQI_total,22.51%,NaN
Tokyo_pm25,22.05%,NaN
Tokyo_pm10,19.84%,NaN


In [ ]:
#Prophetを用いた1日単位の予測モデルの構築
def model_prophet_day(df,city_name):
  for x in ov:
    #ov(object_value)で目的変数として羅列した物事に目的変数名をyとして設定
    df_y = df.rename(columns={x:'y'})
    dff = df_y.rename_axis("ds")
    y_train =dff[["y"]][:-730].reset_index()
    y_val = dff[["y"]][-730:-365].reset_index()
    y_test =dff[["y"]][-365:].reset_index()

    def objective(trial):
      params = {'changepoint_prior_scale' :
                  trial.suggest_uniform('changepoint_prior_scale',
                                        0.001,0.5
                                        ),
                'seasonality_prior_scale' :
                  trial.suggest_uniform('seasonality_prior_scale',
                                        0.01,10
                                        ),
                'seasonality_mode' :
                  trial.suggest_categorical('seasonality_mode',
                                            ['additive', 'multiplicative']
                                            ),
                'changepoint_range' :
                    trial.suggest_discrete_uniform('changepoint_range',
                                                  0.8, 0.95,
                                                  0.001),
                'n_changepoints' :
                    trial.suggest_int('n_changepoints',
                                      20, 35),
              }
      m = Prophet(**params)
      m.fit(y_train)
      df_future = m.make_future_dataframe(periods=len(y_val),freq='D')
      df_pred = m.predict(df_future)
      preds = df_pred.tail(len(y_val))
      val_rmse = np.sqrt(mean_squared_error(y_val.y, preds.yhat))
      return val_rmse


    # ハイパーパラメータの探索の実施
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=100)
    m = Prophet(**study.best_params)
    m.fit(y_train)


    #予測する範囲を検証データとテストデータの二つの長さの合計としてperiodsで指定
    future = m.make_future_dataframe(periods =len(y_val)+len(y_test),freq='D')
    pred = m.predict(future)

    #Prophetによるトレンドと1週間と1年間の季節性の特徴を捉えたグラフの描画・保存
    fig = m.plot_components(pred)
    plt.legend()
    plt.savefig(f'{graph_path}/{city_name}_{x}_prophet_feature.png',bbox_inches='tight')

    #テスト期間のデータのみを取得する
    result_df = pd.merge(pred[["ds","yhat"]].astype(str),y_test.astype(str),on="ds",how="inner")


    #グラフに表示するための
    result_df = result_df.rename(columns={"ds":"Date","y":x,"yhat":"predicted_value"})
    result_df["Date"] = pd.to_datetime(result_df["Date"])
    result_df[x] = result_df[x].astype(float)
    result_df["predicted_value"] = result_df["predicted_value"].astype(float)
    result = result_df.set_index("Date")


    #各目的変数ごとの観測値と予測値のデータフレームをcsvとして保存
    result.to_csv(f"{data_path}/{city_name}_{x}_day.csv")
    #mape値の抽出
    mape_df = str(str(round(mape(result_df["predicted_value"], result_df[x])*100,2))+"%")
    df_evaluation.at[f"{city_name}_{x}","prophet_day"] = mape_df

    #グラフの描画と保存
    result_df.set_index("Date").plot()
    plt.xlabel('Date')
    plt.ylabel(x)
    plt.title(f'{city_name}(2022/11/4~2023/11/3){x}_value_prophet_day')
    plt.legend()
    plt.savefig(f'{graph_path}/{city_name}_{x}_prophet_pred_day.png',bbox_inches='tight')
    plt.show()

  return result

In [ ]:
#Delhi
model_prophet_day(Delhi,"Delhi")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#Tokyo
model_prophet_day(Tokyo,"Tokyo")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
df_evaluation

,prophet_week,prophet_day
Delhi_AQI_total,8.04%,12.87%
Delhi_pm25,14.02%,21.1%
Delhi_pm10,25.82%,34.05%
Delhi_so2,16.9%,19.61%
Delhi_o3,16.01%,22.34%
Delhi_no2,15.72%,19.76%
Delhi_co,15.51%,22.32%
Tokyo_AQI_total,20.94%,35.43%
Tokyo_pm25,27.01%,50.34%
Tokyo_pm10,19.55%,35.43%



# MAPEデータフレームの保存

In [ ]:
#保存場所の指定
mape_path = f"{working_dir}/result/data/MAPE"

In [ ]:
df_evaluation.to_csv(f"{mape_path}/Prophet_mape.csv")